# Balancing Notebook

### Our target variables are quite imbalanced in our dataset. This notebook uses Synthetic Minority Over-sampling Technique (SMOTE) to create new synthetic samples that are similar to existing observations in the minority class. By creating a dataset with balanced target variables we hope to improve model performance and in particular increase the predictive ability on the minority class (experienced fracture). 

#### 1. [Installation and Importing of Libraries](#eda_import)
#### 2. [SMOTE](#eda_smote)
#### 3. [Write a CSV of cleaned data](#eda_csv) (TO-DO: dependent on outcome variable)

### <a name="eda_import"></a>Installation and Importing of Libraries
In order to both explore and visualize the data, it's necessary for us to load various libraries. The first cell installs packages necessary for imblearn.

In [1]:
!pip install cmake

!pip install scikit-learn

!pip install imblearn

In [2]:
##import libraries required for analysis
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler


### <a name="eda_smote"></a>SMOTE

Below is the code to retrieve the clean csv from our merged_data folder.

In [3]:
# Set file path
file_path = "/dsa/groups/casestudy2023su/team03/merged_data/mros_merged_clean.csv"

# Load dataframe from CSV
merged_df = pd.read_csv(file_path)



In [4]:
merged_df.describe()

,B1TRD,B1ITD,B1FND,B1L1D,B1L3D,B1TBD,B1HDD,B1LAD,B1RAD,B1LRD,...,RADIALPULSE_MEAS1,BMI,HEIGHTCHANGEFROM25,WEIGHTCHANGEFROM25,FAFXN,FAFXNT,GIAGE1,FAFXN_BIN,FAFXNT_BIN,outlier
count,5994.000000,5994.000000,5994.000000,5994.000000,5994.000000,5994.000000,5994.000000,5994.000000,5994.000000,5994.000000,...,5994.000000,5994.000000,5994.000000,5994.000000,5994.000000,5994.000000,5994.000000,5994.000000,5994.000000,5994.000000
mean,0.764990,1.111712,0.784170,0.979764,1.098752,1.167219,2.132447,0.848691,0.861511,0.706571,...,64.349891,27.378324,3.705292,10.339891,0.432432,0.343577,73.657658,0.253420,0.214715,0.899900
std,0.127367,0.166538,0.128116,0.176903,0.207320,0.126578,0.332499,0.091894,0.092162,0.135757,...,9.989382,3.829940,2.960090,11.375341,1.002012,0.813297,5.872264,0.435006,0.410659,0.436133
min,0.167047,0.389357,0.272729,0.298691,0.382238,0.764788,1.056920,0.515547,0.515601,0.443625,...,36.000000,17.211000,-30.180000,-42.393700,0.000000,0.000000,64.000000,0.000000,0.000000,-1.000000
25%,0.677239,0.998989,0.696196,0.859821,0.955975,1.083122,1.905628,0.791598,0.804045,0.636937,...,58.000000,24.777850,1.970000,2.994512,0.000000,0.000000,69.000000,0.000000,0.000000,1.000000
50%,0.758539,1.104805,0.773544,0.968030,1.081870,1.157550,2.113950,0.843987,0.856622,0.691454,...,64.000000,26.905750,3.590000,9.365350,0.000000,0.000000,73.000000,0.000000,0.000000,1.000000
75%,0.847302,1.221518,0.860129,1.086195,1.219765,1.241125,2.339317,0.898742,0.913262,0.751444,...,70.000000,29.490625,5.240000,17.125600,1.000000,0.000000,78.000000,1.000000,0.000000,1.000000
max,1.699030,1.984450,1.598350,1.976850,2.245680,2.046580,4.695200,1.945820,2.139360,4.667740,...,198.000000,50.668700,30.870000,69.257700,12.000000,12.000000,100.000000,1.000000,1.000000,1.000000


In [5]:
#split into X and y by dropping target variables from our X variables and using one target at a time for y variable. Also used get_dummies to transform some categorical features.
X = pd.get_dummies(merged_df.drop(['FAFXN', 'FAFXNT', 'FAFXN_BIN', 'FAFXNT_BIN'], axis=1)) 
y = merged_df['FAFXNT_BIN'].astype(int)


In [6]:
# Display the count of each class in the target prior to rebalancing
print(y.value_counts())


0    4707
1    1287
Name: FAFXNT_BIN, dtype: int64


### It is important to split the data into train, test prior to resampling to avoid leakage from the test and validation sets into the training set.

In [7]:
#creating initial temp split of 10% for testing

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
#second split of 10% for validation... leaving us with 80% for training.

X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1111, random_state=42) # 0.1111 * 0.9 = 0.1

In [8]:
#fit smote and create new X and y resampled
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

print(y_res.value_counts())

1    3748
0    3748
Name: FAFXNT_BIN, dtype: int64


## Train a simple logistic regression model to evaluate performance between training data before and after resampling

In [14]:
# Initialize StandardScaler
scaler = StandardScaler()

# Scaling data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_res_scaled = scaler.fit_transform(X_res)

# logistic regression model
model = LogisticRegression(max_iter=1000)

# Fit resampled and scaled training data
model.fit(X_res_scaled, y_res)

# predictions on the test
y_test_pred = model.predict(X_test_scaled)

# evaluate model performance
print("Classification report for resampled training data:")
print(classification_report(y_test, y_test_pred))

# Fit original and scaled training data
model.fit(X_train_scaled, y_train)

# predictions on the test set
y_test_pred_orig = model.predict(X_test_scaled)

# evaluate model performance
print("Classification report for original training data:")
print(classification_report(y_test, y_test_pred_orig))



Classification report for resampled training data:
              precision    recall  f1-score   support

           0       0.91      0.15      0.26       471
           1       0.23      0.95      0.38       129

    accuracy                           0.32       600
   macro avg       0.57      0.55      0.32       600
weighted avg       0.77      0.32      0.29       600

Classification report for original training data:
              precision    recall  f1-score   support

           0       0.80      0.95      0.87       471
           1       0.38      0.12      0.18       129

    accuracy                           0.77       600
   macro avg       0.59      0.53      0.52       600
weighted avg       0.71      0.77      0.72       600

